In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))


This work was done as part of a three month project in collaboration with the Ion Technology group at the University of Sussex.

In this file we are going to describe some basic concepts in quantum compilation, showing how to implement compilation of quantum gates using Qiskit modules, and finally produce a simulated quantum circuit that we have compiled into the native gate set of a trapped ion quantum computer.


# What is classical compilation?

Explaining things in quantum mechanics (or quantum information theory, or quantum computing) is usually made easier by making analogies to the classical case where possible. For that reason, we will begin by understanding classical compilation. 

Compilation is the process of taking code written in a high level programming language (e.g. Python, C++, HTML, etc.) and converting it into the machine language that a computer can 'understand'. At its lowest level, machine language are the 'zeros and ones' that we often hear of.

# What is quantum compilation?

Quantum compilation is similar to classical compilation, in that we take a 'high-level' quantum algorithm (which can be expressed as an arbitrary unitary matrix on $n$ qubits) and decompose it into a set of gates that we can actually perform on a quantum computer.

# Why do we need to compile?

By combining unitaries together using tensory products and sequential multiplication, we can express arbitrary quantum algorithms on $n$ qubits as a single unitary opertor in a $2^{n}$ dimensional Hilbert space. If we can express an arbitrary quantum algorithm as a unitary matrix on $n$ qubits, why can't our quantum computers just implement this gate as it is? The answer is that there is no proposed way to implement gates on anything but one or two qubits. Additionally, quantum computers have very large engineering hurdles to overcome in their current state (only implementing single and two qubit gates), so being able to perform gates on multiple qubits is very unlikely to work. 

Quantum computation can be implemented on different engineering platforms, such as trapped ions, photons or superconducting qubits. Each of these has their own pros and cons, such as coherence time, gate fidelity, and measurement fidelity. They also all have their own native gate sets, which are the set of gates that can actually be performed on them. For example, trapped ion quantum computers can perform $R_{x}(\theta)$, $R_{y}(\phi)$ and the molmer-sorenson two qubit gate, whereas superconducting charge qubits can perform $R_{x}(\theta)$, $R_{z}(\phi)$ and the CNOT two qubit gate. 

We can think of the native gate set as the 'machine language' of the quantum computer. So long as the native gate set is a universal set, then we can approximate any unitary, and therefore any quantum algorithm, on this quantum computer. For a native gate set to be universal, we mean that any unitary can be approximated to arbitrary accuracy using chains of gates in the set.

# What are the different steps of decomposition?

Say we have a quantum algorithm on $n$ qubits that we want to perform on a quantum computer. We know that this algorithm can be written as a unitary matrix belonging to the group $U\in SU(2^{n})$. From [1] we know that this unitary can be *exactly* decomposed into a product of so called two-level unitaries. These are unitaries that only act non-trivially on two basis states. We also know that these two level unitaries can in turn be exactly decomposed into products of arbitrary single qubit gates, and a two qubit entangling gate such as the CNOT. Putting these ideas together, we find out that arbitrary quantum algorithms can be exactly decomposed into products of arbitrary single qubit gates, and two qubit entangling gates.

So if the native gate set of our quantum computer has access to these gates, we are able to *exactly* execute *any* quantum algorithm on it. Unfortunely, no quantum computing platforms have this capability. By 'arbitrary single qubit gates', what we mean is a rotation on the Bloch sphere, by *any* angle, along *any* axis. The best that quantum computing platforms are able to do is rotations on the Bloch sphere by any angle, along two fixed, perpedicular, axes. Thankfully, though, combinations of such gates allow us to approximate the arbitrary single qubits we require.

So as long as our quantum computer has access to two single qubit rotations, of arbitray angle, along two perpendicular axes of the Bloch sphere, and also has access to a two qubit entangling gate, then we can approximate any unitary with it (and therefore execute any quantum algorithm). Such a set of gates is referred to as a universal gate set.

# Tradeoffs in compilation

We can approximate a unitary that we want to implement by chaining together more and more gates from our native gate set. The more gates we use in our approximation the better it will be. If we want to improve the fidelity of our quantum computer, should we just chain together loads and loads of gates from the native gate set, to approximate the unitaries making up the quantum algorithm as closely as possible? No. That idea ignores the fact that each time we phyiscally implement a gate from the native gate set, there is some error associated with it. For this reason, there will be a trade off in the number of gates we use to approximate the unitary that represents the quantum algorithm we want to implement: more gates means that we get a better approximation, but more gates also means more error is accumulated.

# How does our simulator work?

The main goal of this work is to test the fidelity of decomposition. We cannot perform the random two qubit gates used in our quantum volume calculator on a real quantum computer, but we can approximate them using our native gate set. 

In this file, we will run two different quantum circuits using the qiskit library. One will run the random two qubit gates as they are, and the other will decompose them into gates that can actually be performed on a trapped ion quantum computer. We will also add noise to this decomposed circuit, to simulate the errors that qubits in real systems pick up (in the form of decoherence, gate errors, and measurement errors). At the end, we run both circuits (ideal and noisy) and then compare the output states from the two. In this way, we can calculate the fidelity of our decomposed circuit, using the Uhlmann's measure of state fidelity [2]. 

We can then decide on a reasonable fidelity, say 95%, and define the quantum volume as $nm$, provided that the fidelity is above 95%.

We will see below that there are many trade offs to consider when decomposing circuits. We hope that this file will be interesting for users to play around with, and will also provide a good understanding of quantum gate decomposition. Please see the references at the end of the file for an in depth explanation of the concepts in this file. 

In [2]:
# import regular python packages
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import warnings
import scipy.linalg as la
from scipy.stats import unitary_group
from numpy import binary_repr
from numpy.polynomial import Polynomial as P
pi=np.pi


In [3]:
#import Qiskit stuff
from qiskit.circuit.quantumregister import QuantumRegister
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.circuit.library.standard_gates.x import CXGate
from qiskit.circuit.tools import pi_check
from qiskit.exceptions import QiskitError
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info.operators.predicates import is_unitary_matrix
from qiskit.quantum_info.synthesis.weyl import weyl_coordinates
from qiskit.quantum_info.synthesis.two_qubit_decompose import *
from qiskit.quantum_info.synthesis.one_qubit_decompose import *
from qiskit.quantum_info.states.measures import *
from qiskit import(QuantumCircuit,execute,Aer, transpile)
from qiskit.visualization import plot_histogram
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram


ModuleNotFoundError: No module named 'qiskit.quantum_info.synthesis'

In [ ]:
# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
from qiskit.circuit.library.generalized_gates import GMS


In [ ]:
# define useful functions
 
def MolmerSorensen(theta):
    """
    Create a (variable) Molmer-Sorensen two qubit gate
    https://en.wikipedia.org/wiki/M%C3%B8lmer%E2%80%93S%C3%B8rensen_gate

    Args:
        theta: angle of the rotation.
    Returns:
        two qubit molmer-sorensen gate, angle theta
        
    """
    return Operator([
    [np.cos(theta),  0,              0,          -1j*np.sin(theta)],
    [0,             np.cos(theta), -1j*np.sin(theta),             0],
    [0,             -1j*np.sin(theta), np.cos(theta),             0],
    [-1j*np.sin(theta),     0,              0,        np.cos(theta)]])
 
def create_rand_su(n):
    """ 
    Generates a random matrix in the group SU(n)

    Args:
        n: dimension of gate to create
    Returns:
        random matrix in SU(n)
        
    """
    if n==2:
        u_matrix = unitary_group.rvs(2)
        su_matrix = u_matrix/(np.linalg.det(u_matrix)**0.25)
  
    elif n==4:
        u_matrix = unitary_group.rvs(4)
        su_matrix = u_matrix/(np.linalg.det(u_matrix)**0.25)
        
    else:
        print('Please enter n=2 or n=4')
    
    return su_matrix
 

def UJ_fid(A,B):
    """ 
    Calculates the UJ fidelity of two quantum states (matrices)
    https://en.wikipedia.org/wiki/Fidelity_of_quantum_states
    
    Args:
        A: first state
        B: second state
    Returns:
        UJ Fidelity between A and B
        
    """
    return ( np.trace( np.sqrt( np.matmul(np.sqrt(A), np.matmul(B, np.sqrt(A))) ) ) ) **2


def r_X(theta):
    """ 
    Qubit rotation along the x-axis by angle \theta
    
    Args:
        theta: angle of rotation
    Returns:
        Rotation matrix about x axis by theta
        
    """  
    a = math.cos(theta/2.0)
    b = -1j*math.sin(theta/2.0)
    c = -1j*math.sin(theta/2.0)
    d = math.cos(theta/2.0)
    return np.matrix([[a,b],[c,d]])
 
def r_Y(theta):
    """ 
    Qubit rotation along the y-axis by angle \theta
    
    Args:
        theta: angle of rotation
    Returns:
        Rotation matrix about y axis by theta
        
    """    
    a = math.cos(theta/2.0)
    b = -math.sin(theta/2.0)
    c = math.sin(theta/2.0)
    d = math.cos(theta/2.0)
    return np.matrix([[a,b],[c,d]])

def decomp_angles(su2_list, euler_basis='XYX'):
    """  
    Function that takes a list of SU2 matrices and an euler basis and decomposes them all accordingly.
   
    e.g. if euler_basis = XYX" then SU(2) = \exp[i*gamma] Rx(theta) Ry(phi) Rx(xi)
    
    Args:
        su2_list:    list of su2 matrices that we want to decompose 
        euler_basis: the basis that we want to decompose the matrices in su2_list
    Returns:
        theta_list: list of first Euler angles
        phi_list: list of second Euler angles
        xi_list: list of third Euler angles
        gamma_list: list of global phases
        
    """    
    L = len(su2_list)
    angle_list = []
 
    decomposer = OneQubitEulerDecomposer('XYX')
 
    angle_list.extend((decomposer.angles_and_phase(i)) for i in su2_list )
   
    theta_list = [angle_list[i][0] for i in range(L)]
    phi_list   = [angle_list[i][1] for i in range(L)]
    xi_list    = [angle_list[i][2] for i in range(L)]
    gamma_list = [angle_list[i][3] for i in range(L)]
    
    return theta_list, phi_list, xi_list, gamma_list

def create_density_matrix(n_qb, result):
    """  
    Function that creates a density matrix from the measurement result of a quantum circuit
       
    Args:
        n_qb: number of qubits the state is for
        result: measurement result of a quantum circuit
    Returns:
        d_op: density operator reprenting the state at the end of the quantum circuit
        
    """    
    d = np.matrix(np.zeros((2**n_qb,2**n_qb))) # create empty density matrix 
    for i in range(2**n_qb): #loop through diagonal entries  
        bin_i = binary_repr(i, width=n_qb) #get the binary version of the loop counter
        if f"{bin_i}" in result.get_counts(0): 
            a = result.get_counts(0)[f"{bin_i}"]/shot_count #retrieve the counts using the binary number, normalise
            d[i,i] = a #store in the i^th diagonal entry
    d_op = Operator(d) #convert the matrix to an instance of the Operator class
    return d_op



#Define the SWAP operator
swap = Operator([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]])

In [ ]:
### these functions need to be optimized as there is lots of repeated code

def make_circuit(theta, phi, xi, basis_count, n_qb, circuit, m , n):
    """
    Inputs: euler angles from decomposition and the number of basis gates used
    Outputs: quantum circuit using these euler angles, commuting Rx gates through MS gates where possible
    
    """
    n_qubits = n_qb #set the number of qubits

    #INSTEAD OF CREATING A NEW CIRCUIT, WE WANT TO ADD EVERYTHING TO THE NOISY CIRCUIT noisy_qc
    
    if basis_count == 3:
        #build the circuit for the decomposed target
        circuit.rx(xi[6], m)
        circuit.ry(theta[6], m)
        circuit.rx(phi[6] + xi[4], m)
 
        circuit.rx(xi[7], n)
        circuit.ry(theta[7], n)
        circuit.rx(phi[7] + xi[5], n)
 
        circuit.rxx(pi/2, m, n) #MS 3
       
    if basis_count >=2:
        if basis_count ==2:
            circuit.rx(xi[4], m)
        circuit.ry(theta[4], m)
        circuit.rx(phi[4] +xi[2], m)
 
        if basis_count == 2:
            circuit.rx(xi[5], n)
        circuit.ry(theta[5], n)
        circuit.rx(phi[5] + xi[3], n)
 
        circuit.rxx(pi/2, m, n) #MS 2
    if basis_count >=1:
        if basis_count == 1:
            circuit.rx(xi[2], m)
        circuit.ry(theta[2], m)
        circuit.rx(phi[2] +xi[0], m)
 
        if basis_count == 1:
            circuit.rx(xi[3], n)
        circuit.ry(theta[3], n)
        circuit.rx(phi[3] +xi[1], n)
 
        circuit.rxx(pi/2, m, n) #MS 1
    if basis_count >=0:
        if basis_count == 0:
            circuit.rx(xi[0], m)
        circuit.ry(theta[0], m)
        circuit.rx(phi[0], m)
 
        if basis_count == 0:
            circuit.rx(xi[1], n)
        circuit.ry(theta[1], n)
        circuit.rx(phi[1], n)
        
    return




In [ ]:
def recomb0(target, target_kak, su2_fid, basis_fid):
    U0r, U0l = decomp.decomp0(target_kak)
    u0r = np.matrix(U0r)
    u0l = np.matrix(U0l)
    su2_list = [U0l, U0r]
    theta, phi, llambda, gamma = decomp_angles(su2_list, 'XYX')
   
    u0l_euler = np.exp(1j*gamma[0])*np.matmul(r_X(phi[0]), np.matmul(r_Y(theta[0]), r_X(llambda[0])))
    u0r_euler = np.exp(1j*gamma[1])*np.matmul(r_X(phi[1]), np.matmul(r_Y(theta[1]), r_X(llambda[1])))
    A = np.kron(u0l_euler, u0r_euler)
   
    U0 = np.matrix(np.exp(1j*target_kak.global_phase)*A)
    trace0 = np.abs(np.trace(np.matmul(target, U0.H)))
    fid0 = trace_to_fid(trace0)*(su2_fid**3)
 
    return U0, fid0, theta, phi, llambda, gamma


In [ ]:

def recomb1(target, target_kak, su2_fid, basis_fid):
    """
    Inputs: target SU(4) and its KAK decomposition class
    Outputs: the approximated matrix using 1 basis gates, and the fidelity of this approximation
    Fidelity calcualted by recombining the decomposition products and calculating Tr[target x approximation]
 
    """
    U1r, U1l, U0r, U0l = decomp.decomp1(target_kak)
    #turn products to np.matrix class
    u1r = np.matrix(U1r)
    u1l = np.matrix(U1l)
    u0r = np.matrix(U0r)
    u0l = np.matrix(U0l)
 
     #find the decomposition angles of the SU(2)s in the decomposition
    su2_list = [u0l, u0r, u1l, u1r]
    theta, phi, llambda, gamma = decomp_angles(su2_list, 'XYX')
    u0l_euler = np.exp(1j*gamma[0])*np.matmul(r_X(phi[0]), np.matmul(r_Y(theta[0]), r_X(llambda[0])))
    u0r_euler = np.exp(1j*gamma[1])*np.matmul(r_X(phi[1]), np.matmul(r_Y(theta[1]), r_X(llambda[1])))
    u1l_euler = np.exp(1j*gamma[2])*np.matmul(r_X(phi[2]), np.matmul(r_Y(theta[2]), r_X(llambda[2])))
    u1r_euler = np.exp(1j*gamma[3])*np.matmul(r_X(phi[3]), np.matmul(r_Y(theta[3]), r_X(llambda[3])))
 
    #recompose entire matrix from products
    # U = exp[iphase] (U0L⊗U0R) CNOT (U1L⊗U1R)
    A = np.kron(u0l_euler, u0r_euler)
    B = np.kron(u1l_euler, u1r_euler)
    #need basis_gate.data since basis_gate is type(operator) not matrix
    U1 = np.matrix(np.exp(1j*target_kak.global_phase) * np.matmul(A, np.matmul(basis_gate.data,B)))
    trace1 = np.abs(np.trace(np.matmul(target, U1.H)))
    fid1 = trace_to_fid(trace1)*(su2_fid**5)*(basis_fid)
    return U1, fid1, theta, phi, llambda, gamma
 

In [ ]:

def recomb2(target, target_kak, su2_fid, basis_fid):
    """
    Inputs: target SU(4) and its KAK decomposition class
    Outputs: the approximated matrix using 2 basis gates, and the fidelity of this approximation
    Fidelity calcualted by recombining the decomposition products and calculating Tr[target x approximation]
 
    """
    U2r, U2l, U1r, U1l, U0r, U0l = decomp.decomp2_supercontrolled(target_kak)
    #turn products to np.matrix class
    u2r = np.matrix(U2r)
    u2l = np.matrix(U2l)
    u1r = np.matrix(U1r)
    u1l = np.matrix(U1l)
    u0r = np.matrix(U0r)
    u0l = np.matrix(U0l)
   
    #find the decomposition angles of the SU(2)s in the decomposition
    su2_list = [u0l, u0r, u1l, u1r ,u2l, u2r]
    theta, phi, llambda , gamma = decomp_angles(su2_list, 'XYX')
    u0l_euler = np.exp(1j*gamma[0])*np.matmul(r_X(phi[0]), np.matmul(r_Y(theta[0]), r_X(llambda[0])))
    u0r_euler = np.exp(1j*gamma[1])*np.matmul(r_X(phi[1]), np.matmul(r_Y(theta[1]), r_X(llambda[1])))
    u1l_euler = np.exp(1j*gamma[2])*np.matmul(r_X(phi[2]), np.matmul(r_Y(theta[2]), r_X(llambda[2])))
    u1r_euler = np.exp(1j*gamma[3])*np.matmul(r_X(phi[3]), np.matmul(r_Y(theta[3]), r_X(llambda[3])))
    u2l_euler = np.exp(1j*gamma[4])*np.matmul(r_X(phi[4]), np.matmul(r_Y(theta[4]), r_X(llambda[4])))
    u2r_euler = np.exp(1j*gamma[5])*np.matmul(r_X(phi[5]), np.matmul(r_Y(theta[5]), r_X(llambda[5])))
    #recompose entire matrix from products
    #U = exp[iphase] (U0L⊗U0R) CNOT (U1L⊗U1R) CNOT (U2L⊗U2R)
    A = np.kron(u0l_euler,u0r_euler)
    B = np.kron(u1l_euler,u1r_euler)
    C = np.kron(u2l_euler,u2r_euler)
    U2 = np.matrix(np.exp(1j*target_kak.global_phase)*np.matmul(A, np.matmul(basis_gate.data, np.matmul(B, np.matmul(basis_gate.data, C)))))
    trace2 = np.abs(np.trace(np.matmul(target, U2.H)))
    fid2 = trace_to_fid(trace2)*(su2_fid**7)*(basis_fid**2)
    return U2, fid2, theta, phi, llambda, gamma


In [ ]:

def recomb3(target, target_kak, su2_fid, basis_fid):
    """
    Inputs: target SU(4) and its KAK decomposition class
    Outputs: the approximated matrix using 3 basis gates, and the fidelity of this approximation
    Fidelity calcualted by recombining the decomposition products and calculating Tr[target x approximation]
 
    """
    U3r, U3l, U2r, U2l, U1r, U1l, U0r, U0l = decomp.decomp3_supercontrolled(target_kak)
    #turn products to np.matrix class
    u3r = np.matrix(U3r)
    u3l = np.matrix(U3l)
    u2r = np.matrix(U2r)
    u2l = np.matrix(U2l)
    u1r = np.matrix(U1r)
    u1l = np.matrix(U1l)
    u0r = np.matrix(U0r)
    u0l = np.matrix(U0l)
    #find the decomposition angles of the SU(2)s in the decomposition
    su2_list = [u0l, u0r, u1l, u1r ,u2l, u2r, u3l, u3r]
    theta, phi, llambda, gamma = decomp_angles(su2_list, 'XYX')
    #recompose entire matrix from products
    # U = exp[iphase] (U0L⊗U0R) CNOT (U1L⊗U1R) CNOT (U2L⊗U2R) CNOT (U3L⊗U3R)
   
    u0l_euler = np.exp(1j*gamma[0])*np.matmul(r_X(phi[0]), np.matmul(r_Y(theta[0]), r_X(llambda[0])))
    u0r_euler = np.exp(1j*gamma[1])*np.matmul(r_X(phi[1]), np.matmul(r_Y(theta[1]), r_X(llambda[1])))
    u1l_euler = np.exp(1j*gamma[2])*np.matmul(r_X(phi[2]), np.matmul(r_Y(theta[2]), r_X(llambda[2])))
    u1r_euler = np.exp(1j*gamma[3])*np.matmul(r_X(phi[3]), np.matmul(r_Y(theta[3]), r_X(llambda[3])))
    u2l_euler = np.exp(1j*gamma[4])*np.matmul(r_X(phi[4]), np.matmul(r_Y(theta[4]), r_X(llambda[4])))
    u2r_euler = np.exp(1j*gamma[5])*np.matmul(r_X(phi[5]), np.matmul(r_Y(theta[5]), r_X(llambda[5])))
    u3l_euler = np.exp(1j*gamma[6])*np.matmul(r_X(phi[6]), np.matmul(r_Y(theta[6]), r_X(llambda[6])))
    u3r_euler = np.exp(1j*gamma[7])*np.matmul(r_X(phi[7]), np.matmul(r_Y(theta[7]), r_X(llambda[7])))
 
    A = np.kron(u0l_euler,u0r_euler)
    B = np.kron(u1l_euler,u1r_euler)
    C = np.kron(u2l_euler,u2r_euler)
    D = np.kron(u3l_euler,u3r_euler)
    U3 = np.matrix (np.exp(1j*target_kak.global_phase)*np.matmul(A, np.matmul(basis_gate.data, np.matmul(B, np.matmul(basis_gate.data, np.matmul(C, np.matmul(basis_gate.data, D)))))))
    trace3 = np.abs(np.trace(np.matmul(target, U3.H)))
    fid3 = trace_to_fid(trace3)*(su2_fid**9)*(basis_fid**3)
    return U3, fid3, theta, phi, llambda, gamma


In [ ]:
### DECOMPOSITION BASIS (TRAPPED IONS) ###

### Choose the two qubit basis gate used in KAK decomposition
basis_gate = MolmerSorensen(pi/4)
basis_gate_label = 'MS'

### Choose single qubit basis gates to be used in KAK decomposition
euler_basis = 'XYX' #e.g. "XYX" turns an SU(2) into Rx(phi)Ry(theta)Rx(llambda)



In [ ]:
# ### DECOMPOSITION BASIS (SUPERCONDUCTING QUBITS) ###

# ### Choose the two qubit basis gate used in KAK decomposition
# basis_gate = cx()
# basis_gate_label = 'CNOT'

# ### Choose single qubit basis gates to be used in KAK decomposition
# euler_basis = 'XZX' #e.g. "XZX" turns an SU(2) into Rx(phi)Rz(theta)Rx(llambda)



In [ ]:
#Function for fidelity to probability of error conversion

#single qubit gate data
x = [0.00000927357382,0.00001260986452,0.00002928106781,0.00003274528310,0.00004924902626,0.00006055222635,0.00011460445423,0.00016071899407,0.00022938562612,0.00027801912678,0.00030185445510,0.00043266624068,0.00044177729723,0.00057279002193,0.00057682838342,0.00070890425941,0.00077947268436,0.00079515532825,0.00080682762917,0.00085089608318,0.00100219273868,0.00104089382283,0.00109095947198,0.00104372058209,0.00116174830392,0.00123813079998,0.00129937860462,0.00128349409828,0.00130493212361,0.00139111136819,0.00145414403424,0.00154043057681,0.00153916316729,0.00176066840966,0.00213276093073,0.00245376808135,0.00265083092714,0.00265166760575,0.00324197197114,0.00326583722022,0.00320380389140,0.00328885321692,0.00348062433166,0.00347240388496,0.00323130829847,0.00344292847993,0.00356279994437,0.00383968605530,0.00396580149891,0.00433704004305,0.00426199843599,0.00398851122681,0.00420891018456,0.00397290379810,0.00430589372375,0.00449442198642,0.00528310345302,0.00592808713104,0.00606595182665,0.00596611750736,0.00626052406424,0.00596401086915,0.00559063279007,0.00570511992288,0.00575084539024,0.00610864910472,0.00612002018331,0.00711666299345,0.00719744952783,0.00676697535723,0.00683955664349,0.00737435778289,0.00770993945309,0.00738052359704,0.00789639090326,0.00879793044710,0.00920989799069,0.00917868385183,0.00939672082347,0.00923738673725,0.00861016938581,0.00834737700795,0.00786299725598,0.00813393990978,0.00851132102235,0.00961866303807,0.01016420541433,0.01134483981584,0.01215935006530,0.01228410946338,0.01239877667888,0.01247468051176,0.01314578489714,0.01271362217630,0.01259739901016,0.01293545345938,0.01223001241374]
y = [0,0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02,0.021,0.022,0.023,0.024,0.025,0.026,0.027,0.028,0.029,0.03,0.031,0.032,0.033,0.034,0.035,0.036,0.037,0.038,0.039,0.04,0.041,0.042,0.043,0.044,0.045,0.046,0.047,0.048,0.049,0.05,0.051,0.052,0.053,0.054,0.055,0.056,0.057,0.058,0.059,0.06,0.061,0.062,0.063,0.064,0.065,0.066,0.067,0.068,0.069,0.07,0.071,0.072,0.073,0.074,0.075,0.076,0.077,0.078,0.079,0.08,0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09,0.091,0.092,0.093,0.094,0.095,0.096]

polynomial_single = P.fit(x, y, 10) # 14 is the degree of the polynomial

#two qubit gate data
x = [2.74212E-05,6.74248E-05,0.000152449,0.000257546,0.000388857,0.000545713,0.000941022,0.000879523,0.001199727,0.001450736,0.00191575,0.001881703,0.002287163,0.002890946,0.003120413,0.003645147,0.003690388,0.004128885,0.004342591,0.004629127,0.005132646,0.006086767,0.00615817,0.006646223,0.007228772,0.006925123,0.008122698,0.008420623,0.008606475,0.008097651,0.009887001,0.010068802,0.010989772,0.011232219,0.010628959,0.012174436,0.011912745,0.012518089,0.014012194,0.013461999,0.014363502,0.015807069,0.014238445,0.016843228,0.016473268,0.016262249,0.017594919,0.018297332,0.018997607,0.018163527,0.019728193,0.021631942,0.020939053]
y = [0,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.0011,0.0012,0.0013,0.0014,0.0015,0.0016,0.0017,0.0018,0.0019,0.002,0.0021,0.0022,0.0023,0.0024,0.0025,0.0026,0.0027,0.0028,0.0029,0.003,0.0031,0.0032,0.0033,0.0034,0.0035,0.0036,0.0037,0.0038,0.0039,0.004,0.0041,0.0042,0.0043,0.0044,0.0045,0.0046,0.0047,0.0048,0.0049,0.005,0.0051,0.0052]

polynomial_double = P.fit(x, y, 10) # 14 is the degree of the polynomial


In [ ]:
#---------------------------SET CIRCUIT DIMENSIONS---------------------------#

n_qb=2 #choose number of qubits
depth=1 #choose depth of circuit


In [ ]:
#---------------------------SET ERROR PARAMETERS---------------------------#
#Valid range: 98-99.99%
two_qubit_gate_fid = 99.9

#Valid range: 99-99.999
single_qubit_gate_fid = 99.99

su2_fid=single_qubit_gate_fid/100
basis_fid=two_qubit_gate_fid/100


#Coherence times [ns]
t1 = 2e12
t2 = 2e9

#Instruction times [ns]
time_shuttle = 5e6

#decomposition_method='exact'      # this method will always decompose the two qubit gates exactly (using three instances of the two qubit basis gate)
decomposition_method='approximate' # this method tries decomposing the circuit using 1,2 and 3 instances of the two qubit basis gate, and chooses which has the best fidelity 

# - there is a tradeoff between more two qubit gates giving a better approximation to the random gates, but also introducing more error

shot_count = 1e6 #number of shots for the simulations (max = 1000000)

#create the class needed to decompose the 'A' from KAK
decomp = TwoQubitBasisDecomposer(basis_gate.data, two_qubit_gate_fid, euler_basis)

qb_label = np.arange(0, n_qb).tolist() #create list of qubit labels

#create noise models, including the gates we want to add noise to
total_noise_model = NoiseModel(['rz', 'rx', 'ry', 'rxx']) #thermal and operation noise



In [ ]:
### add noise to the circuit

#---------------------------THERMAL NOISE---------------------------#

#Empty lists to store single and two qubit errors in
errors_reset   = []
errors_measure = []
errors_rx      = []
errors_ry      = []
errors_shuttle = []
errors_rxx     = []

#create distribution to sample coherence times from
T1s = np.random.normal(t1, t1*0.2, n_qb)
T2s = np.random.normal(t2, t2*0.2, n_qb)
#Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(n_qb)])

#create decoherence errors
errors_shuttle  = [thermal_relaxation_error(t1, t2, time_shuttle)
              for t1, t2 in zip(T1s, T2s)]

#add errors to noise model
for j in range(n_qb): #loop through each qubit
    # decoherence error to shuttle operation (rz(0))
    total_noise_model.add_quantum_error(errors_shuttle[j], "rz", [j])

#---------------------------OPERATION NOISE---------------------------#

#operation based errors:
p_error_single = polynomial_single(1-single_qubit_gate_fid/100)
p_error_double = polynomial_double(1-two_qubit_gate_fid/100)

op_error_single = pauli_error([('X', p_error_single), ('Y', p_error_single), ('Z', p_error_single), ('I', 1 - 3*p_error_single)]) #equal chance of each pauli gate
op_error_double = pauli_error([('X', p_error_double), ('Y', p_error_double), ('Z', p_error_double), ('I', 1 - 3*p_error_double)])
op_error2 = op_error_double.tensor(op_error_double) #tensor the error operation with itself to be used in 2qb gate

for j in np.arange(n_qb):
    #single qubit operation errors
    total_noise_model.add_quantum_error(op_error_single, "rx", [j])
    total_noise_model.add_quantum_error(op_error_single, "ry", [j])
    
    for k in np.arange(n_qb):
        #two qubit errors
        if j != k: 
            #only choose two qubit errors for different qubits
            total_noise_model.add_quantum_error(op_error2, "rxx", [j, k])

print("Two qubit gate fidelity is:", two_qubit_gate_fid,"%.", "Single qubit gate fidelity is:",single_qubit_gate_fid,"%")
print("Also adding error for decoherence during shuttling...")
print('...Finished adding errors to noise model')

In [ ]:
#create ideal quantum circuit (no decomposition, no noise)
qc = QuantumCircuit(n_qb,n_qb)

#create noisy quantum circuit (decomposition and noise)
noisy_qc = QuantumCircuit(n_qb,n_qb)

In [ ]:
### run the calculator

if decomposition_method == "exact":
    x=3
elif decomposition_method == "approximate":
    x=None
else:
    print('invalid input')

In [ ]:
#loop through depth of circuit
for d in range(depth):
    #for each layer in the depth
    
    #create random list of ints from 1 to n to use in pairing qubits randomly
    random.shuffle(qb_label)
    #print()
    #print(qb_label)
    for m in np.arange(0,n_qb): 
        #loop through all qubits (in both circuits) and add an identity (for shuttling)
        #(assumes that there is shuttling inbetween each layer of depth)
        
        #qc.id(m) #shuttle each qubit in the ideal circuit
        noisy_qc.rz(0,m) #shuttle each qubit in the noisy circuit
        
    for n in np.arange(0,n_qb,2):
        #for each (random) pair of qubits create a random two qubit gate, 
        #if the number of qubits is odd then choose one of the qubits to have the identity
        
        #if there are an odd number of qubits, add an identity to the last one then...
        if len(qb_label) % 2 == 1 and n == n_qb-1:
            print(f'qb {qb_label[n]} gets the identity')
            qc.id(qb_label[n])
            noisy_qc.id(qb_label[n])
            
        #otherwise, pair qubits together and create a random SU(4) for them
        else:
            print(f'pairing qubits {qb_label[n]} and {qb_label[n+1]}')
            
            #for each (random) pair of qubits create a random two qubit gate, 

            #create random su4
            rand_su4 = create_rand_su(4)
            target = Operator(rand_su4)
            

            #multiply the target by the SWAP matrix and store in a separate variable
            target_swap = Operator(np.matmul(swap.data, rand_su4))

            #decompose the target using...
            target_kak = TwoQubitWeylDecomposition(target.data)
            U0, fid0, theta0, phi0, xi0, gamma0 = recomb0(target.data, target_kak, su2_fid, basis_fid) # ...0 basis gates
            U1, fid1, theta1, phi1, xi1, gamma1 = recomb1(target.data, target_kak, su2_fid, basis_fid) # ...1 basis gate
            U2, fid2, theta2, phi2, xi2, gamma2 = recomb2(target.data, target_kak, su2_fid, basis_fid) # ...2 basis gates
            U3, fid3, theta3, phi3, xi3, gamma3 = recomb3(target.data, target_kak, su2_fid, basis_fid) # ...3 basis gates

            #decompose the SWAP*target using...
            target_kak_swap = TwoQubitWeylDecomposition(target_swap.data)
            U0_swap, fid0_swap, theta0_swap, phi0_swap, xi0_swap, gamma0_swap = recomb0(target_swap.data, target_kak_swap, su2_fid, basis_fid) #...0 basis gates
            U1_swap, fid1_swap, theta1_swap, phi1_swap, xi1_swap, gamma1_swap = recomb1(target_swap.data, target_kak_swap, su2_fid, basis_fid) #...1 basis gate
            U2_swap, fid2_swap, theta2_swap, phi2_swap, xi2_swap, gamma2_swap = recomb2(target_swap.data, target_kak_swap, su2_fid, basis_fid) #...2 basis gates
            U3_swap, fid3_swap, theta3_swap, phi3_swap, xi3_swap, gamma3_swap = recomb3(target_swap.data, target_kak_swap, su2_fid, basis_fid) #...3 basis gates

            #store the 'operator fidelity' values in a list to choose the best decomposition to use
            fid_list = [fid0, fid1, fid2, fid3, fid0_swap, fid1_swap, fid2_swap, fid3_swap ] 
            if x != 3: #if the user didn't enter '3' (that they want to always use the exact method)...
                x = np.argmax(fid_list) #...choose the decomposition that gives the highest operator fidelity
            #if the SWAP fidelity is the same as without a SWAP, choose normal (no point in doing an unnecessary SWAP)
            if x == 7 and fid_list[7] == fid_list[3]: 
                x = 3
            elif x == 6 and fid_list[6] == fid_list[2]:
                x = 2
            elif x == 5 and fid_list[5] == fid_list[1]:
                x = 1
            elif x == 4 and fid_list[4] == fid_list[0]:
                x = 0
            #print(f'x={x}')
        
            if x >= 4: #if the max fid is one from the SWAP*target decomps, add a SWAP to the start of the circuit
                print(f'SWAP between {qb_label[n]} and {qb_label[n+1]}')
                #qc.swap(qb_label[n],qb_label[n+1])
                #noisy_qc.swap(qb_label[n],qb_label[n+1])
            
            #add the target to the ideal circuit
            qc.unitary(rand_su4, [qb_label[n+1], qb_label[n]], label=f'U{qb_label[n], qb_label[n+1]}')
                 
            #...pair off all pairs of qubits (works for odd and even number of qubits)
            if x==0:
                make_circuit(theta0, phi0, xi0, 0, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==1:
                make_circuit(theta1, phi1, xi1, 1, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==2:
                make_circuit(theta2, phi2, xi2, 2, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==3:
                make_circuit(theta3, phi3, xi3, 3, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            #SWAP*target
            elif x==4:
                make_circuit(theta0_swap, phi0_swap, xi0_swap, 0, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==5:
                make_circuit(theta1_swap, phi1_swap, xi1_swap, 1, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==6:
                make_circuit(theta2_swap, phi2_swap, xi2_swap, 2, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            elif x==7:
                make_circuit(theta3_swap, phi3_swap, xi3_swap, 3, n_qb, noisy_qc, qb_label[n], qb_label[n+1])
            else:
                print('somethings gone wrong')
            

#measure the qubits in both circuits
for n in range(n_qb):
    qc.measure(n,n)
    noisy_qc.measure(n,n)

#run the noisey circuit and extract the result
job_ideal = execute(qc, QasmSimulator(), shots=shot_count)
result_ideal = job_ideal.result()
counts_ideal = result_ideal.get_counts(0)


#run the noisey circuit and extract the result
job_noisy = execute(noisy_qc, QasmSimulator(noise_model=total_noise_model), shots=shot_count)
result_noisy = job_noisy.result()
counts_noisy = result_noisy.get_counts(0)



In [ ]:
print(qc)
print()
print(noisy_qc)

In [ ]:
## calculate fidelity and plot histogram

#exact density matrix that we would get if we could implement arbitrary two qubit gates
rho = create_density_matrix(n_qb, result_ideal) 

#density matrix we get after decomposing the arb two qubit gate into the native gate set and add physical noise to the circuit
sigma = create_density_matrix(n_qb, result_noisy) 

fid = UJ_fid(rho.data, sigma.data).real

legend = ['Ideal', 'Noisy Decomp']

plot_histogram([counts_ideal, counts_noisy], figsize=(10,8), legend=legend, color=['crimson','midnightblue'],
                title=f'Fidelity = {fid:.10f}')

# References
[1] Nielsen, M. A., & Chuang, I. (2002). Quantum computation and quantum information. 

[2] https://en.wikipedia.org/wiki/Fidelity_of_quantum_states#Uhlmann's_theorem